Code mining partie PS 

In [ ]:
from pynq import Overlay

SHA256 = Overlay('/home/xilinx/jupyter_notebooks/DJ/dj.bit')

ip=SHA256.sha256_0
mmio=ip.mmio
register_map=ip.register_map
registers=register_map._register_classes